In [ ]:
"""
Test script for connecting to Triton server and running inference with YOLOV11
"""

import time
import numpy as np
from PIL import Image
import requests
from ultralytics import YOLO

# Constants
TRITON_SERVER_URL = "triton_server:8000"
MODEL_NAME = "chest_xray_detector"
TEST_IMAGE_PATH = "/path/to/test_image.jpg"  # Update with your test image path

In [ ]:
def check_server_status():
    """Check if Triton server is running and model is ready"""
    try:
        # Check server health
        health_url = f"http://{TRITON_SERVER_URL}/v2/health/ready"
        response = requests.get(health_url)
        if response.status_code == 200:
            print("✅ Triton server is ready")
            
            # Check model status
            model_url = f"http://{TRITON_SERVER_URL}/v2/models/{MODEL_NAME}"
            model_response = requests.get(model_url)
            if model_response.status_code == 200:
                print(f"✅ Model '{MODEL_NAME}' is available")
                return True
            else:
                print(f"❌ Model '{MODEL_NAME}' not found or not ready")
                return False
        else:
            print("❌ Triton server is not ready")
            return False
    except Exception as e:
        print(f"Error connecting to server: {e}")
        return False

def test_ultralytics_triton():
    """Test using the Ultralytics YOLO with Triton"""
    print("\nTesting Ultralytics YOLO with Triton...")
    
    try:
        # Load the Triton Server model - exactly as shown in the documentation
        model = YOLO(f"http://{TRITON_SERVER_URL}/{MODEL_NAME}", task="detect")
        print("Model loaded successfully")
        
        # Run inference on a test image
        if TEST_IMAGE_PATH and TEST_IMAGE_PATH != "/path/to/test_image.jpg":
            start_time = time.time()
            results = model(TEST_IMAGE_PATH)
            inference_time = time.time() - start_time
            
            print(f"Inference completed in {inference_time:.4f} seconds")
            
            # Display results
            for i, result in enumerate(results):
                boxes = result.boxes
                print(f"Found {len(boxes)} detections:")
                
                for j, box in enumerate(boxes):
                    cls = int(box.cls[0])
                    cls_name = result.names[cls] if cls in result.names else f"Class {cls}"
                    conf = float(box.conf[0])
                    print(f"  Detection {j+1}: {cls_name} (Confidence: {conf:.3f})")
        else:
            print("No test image path provided. Skipping inference test.")
    
    except Exception as e:
        print(f"Error during testing: {e}")
        print("\nTroubleshooting tips:")
        print("1. Check that Triton server is running (try the check_server_status function)")
        print("2. Verify that the model name is correct")
        print("3. Make sure the model is properly loaded in Triton (check server logs)")
        print("4. Ensure the test image path is correct")


In [ ]:
# Check server status
if check_server_status():
    # Run test with Ultralytics
    test_ultralytics_triton()
else:
    print("Server check failed. Please ensure Triton server is running.")